## Installing Libraries

In [ ]:
!pip install datasets
! pip install -U accelerate==0.15.0
! pip install -U transformers==4.28.1

  Using cached numpy-1.26.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pandas-2.1.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached aiohttp-3.9.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.4 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached multidict-6.0.4-cp39-cp39-macosx_11_0_arm64.whl (29 kB)
  Using cached yarl-1.9.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached frozenlist-1.4.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached 

In [ ]:
#!pip install matplotlib
#!pip install tensorflow
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp39-cp39-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.11.4-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp39-cp39-macosx_12_0_arm64.whl (9.5 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached scipy-1.11.4-cp39-cp39-macosx_12_0_arm64.whl (29.7 MB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments,DistilBertTokenizerFast
from transformers import AutoTokenizer,TFTrainingArguments
from transformers import OpenAIGPTForSequenceClassification, OpenAIGPTTokenizer,OpenAIGPTModel
from datasets import load_dataset
from torch.utils.data import Dataset
import torch
import tensorflow as tf

/Users/bhanuvardhan/Documents/naturalLanguageProcessing/openai-gpt/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bhanuvardhan/Documents/naturalLanguageProcessing/openai-gpt/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = pd.read_csv('data/gamedev.csv')
df.drop('id', axis=1).head()

,user_id,tags,question,answer
0,4450,"['pause', 'timescale']",How can I pause my game?,"In the Editor, you can just click the pause bu..."
1,4450,"['pause', 'timescale']",What is the best way to pause my game?,"In the Editor, you can just click the pause bu..."
2,4450,"['pause', 'timescale']","When I play a game, how do I pause it?","In the Editor, you can just click the pause bu..."
3,4450,"['pause', 'timescale']",Could you please tell me how I can pause my game?,"In the Editor, you can just click the pause bu..."
4,4450,"['pause', 'timescale']","In order to pause my game, what should I do?","In the Editor, you can just click the pause bu..."


## Data Preparation

In [ ]:
X = df['question'][:1000]
y = df['answer'][:1000]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=41)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=41)

y_train_id2label = {id:label for id, label in enumerate(y_train)}
y_train_label2id = {label:id for id, label in enumerate(y_train)}

y_eval_id2label = {id:label for id, label in enumerate(y_eval)}
y_eval_label2id = {label:id for id, label in enumerate(y_eval)}

y_test_id2label = {id:label for id, label in enumerate(y_test)}
y_test_label2id = {label:id for id, label in enumerate(y_test)}

## Tuning

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

# Define your fine-tuning dataset class
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()

        return {'input_ids': input_ids}

# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can also use other GPT-2 variants like "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# Define your fine-tuning dataset
train_texts = X_train.to_list()
train_dataset = CustomDataset(train_texts, tokenizer, max_length=128)

# Create DataLoader for training
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Fine-tune the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        inputs = batch['input_ids'].to(device)

        # Labels are shifted by one position
        labels = inputs.clone()
        labels[labels[:, :] == tokenizer.pad_token_id] = -100
        labels = labels[:, 1:]

        inputs = inputs[:, :-1]

        print("Input Shape:", inputs.shape)
        print("Labels shape: ", labels.shape)

        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_gpt2')
tokenizer.save_pretrained('fine_tuned_gpt2')


Epoch 1/2:   0%|          | 0/90 [00:00<?, ?it/s]

Input Shape: torch.Size([8, 127])
Labels shape:  torch.Size([8, 127])


Epoch 1/2:   1%|          | 1/90 [00:02<03:24,  2.30s/it]

Input Shape: torch.Size([8, 127])
Labels shape:  torch.Size([8, 127])


Epoch 1/2:   2%|▏         | 2/90 [00:04<03:00,  2.05s/it]

...

Epoch 2/2:  99%|█████████▉| 89/90 [02:21<00:01,  1.83s/it]

Input Shape: torch.Size([8, 127])
Labels shape:  torch.Size([8, 127])


Epoch 2/2: 100%|██████████| 90/90 [02:22<00:00,  1.58s/it]


Epoch 2/2, Average Loss: 2.773052971892887


('fine_tuned_gpt2/tokenizer_config.json',
 'fine_tuned_gpt2/special_tokens_map.json',
 'fine_tuned_gpt2/vocab.json',
 'fine_tuned_gpt2/merges.txt',
 'fine_tuned_gpt2/added_tokens.json')

## Testing the model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model and tokenizer
fine_tuned_model = GPT2LMHeadModel.from_pretrained('fine_tuned_gpt2')
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained('fine_tuned_gpt2')

# Set the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fine_tuned_model.to(device)

# Function to generate text
def generate_text(prompt, model, tokenizer, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "when i play a game how do i pause it?"
generated_text = generate_text(prompt, fine_tuned_model, fine_tuned_tokenizer, max_length=20)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


when i play a game how do i pause it? don know to how stop? it being?


Problems:

Not able to train with complete data.
That's the reason prediction in not that accurate